# Flight Delay Project

### Description
In this notebook we will prepare airline and weather data in order to eventually create a model that can flag potential flights for a higher chance of being delayed. A flight will be considered delayed if it is more than 8 minutes past its expected arrival time. 

### Import Modules
<ul>
<li>You will first need to install a number of modules in order to follow along with this notebook. 
<li>Most of these packages, such as numpy and pandas, are available using <a href="https://conda.io/docs/user-guide/install/index.html">Anaconda</a>. 
<li>For the machine learning pipeline, we will be making use of the <a href="https://bigml.readthedocs.io/en/latest/">BigML Python bindings</a>.

In [1]:
import pandas as pd
import numpy as np

#This option lets us view all columns of our dataset
pd.set_option('display.max_columns', 500)

#We will ignore warning flags in the code
import warnings
warnings.filterwarnings('ignore')

In [2]:
import sys
print(sys.version)

3.6.5 |Anaconda, Inc.| (default, Apr 26 2018, 08:42:37) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


In [3]:
df = pd.read_csv("data/airline_data.csv") 

### We will remove columns which are not used in analysis and delete the categorized delay variables because many instances do not contain the data.

In [4]:
df.drop(['ORIGIN_STATE_ABR','ORIGIN_AIRPORT_ID','DEST_AIRPORT_ID','DEST_STATE_ABR'],axis=1,inplace=True)
#Delete the categorized delay variables because less than 10% records have these data. Rest are NaNs
df.drop(['Unnamed: 27','CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY','SECURITY_DELAY','LATE_AIRCRAFT_DELAY'],axis=1,inplace=True)

### We will drop the rows with missing values in important columns, such as departure times.

In [5]:
#Drop rows with missing data in the important columns, i.e. the predictors for flight delay
total_data_rows = len(df.index)
#Drop NaNs
df.dropna(subset = ['UNIQUE_CARRIER','ORIGIN','DEST','CRS_DEP_TIME','CRS_ARR_TIME','ARR_DELAY','CRS_ELAPSED_TIME','DISTANCE'],inplace=True)
data_retained = len(df.index)/total_data_rows
print('Data Retained: '+str(round(data_retained*100,2))+' %')

Data Retained: 98.93 %


In [6]:
df.head(2)

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,UNIQUE_CARRIER,FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE
0,2013,9,2,1,9E,3283,AUS,SLC,1420,1416.0,-4.0,1610,1552.0,-18.0,170.0,156.0,136.0,1086.0
1,2013,9,3,2,9E,3283,BUF,JFK,1350,1345.0,-5.0,1525,1522.0,-3.0,95.0,97.0,65.0,301.0


### We will filter on the top 50 airports (sorted by air traffic), contained in a given CSV file.

In [7]:
top50_airport = pd.read_csv('data/top50airports.csv')['IATA'].tolist()
df = df[df['ORIGIN'].isin(top50_airport)]
df = df[df['DEST'].isin(top50_airport)]

###  We will now join our weather data that was extracted from www.ncdc.noaa.gov.in

In [8]:
df_weather = pd.read_csv("data/860638.csv")
df_weather = df_weather.append(pd.read_csv('data/860640.csv'))

In [9]:
df_weather.head(5)

,STATION,STATION_NAME,ELEVATION,LATITUDE,LONGITUDE,DATE,REPORTTPYE,HOURLYSKYCONDITIONS,HOURLYVISIBILITY,HOURLYPRSENTWEATHERTYPE,HOURLYDRYBULBTEMPF,HOURLYDRYBULBTEMPC,HOURLYWETBULBTEMPF,HOURLYWETBULBTEMPC,HOURLYDewPointTempF,HOURLYDewPointTempC,HOURLYRelativeHumidity,HOURLYWindSpeed,HOURLYWindDirection,HOURLYWindGustSpeed,HOURLYStationPressure,HOURLYPressureTendency,HOURLYPressureChange,HOURLYSeaLevelPressure,HOURLYPrecip,HOURLYAltimeterSetting,DAILYMaximumDryBulbTemp,DAILYMinimumDryBulbTemp,DAILYAverageDryBulbTemp,DAILYDeptFromNormalAverageTemp,DAILYAverageRelativeHumidity,DAILYAverageDewPointTemp,DAILYAverageWetBulbTemp,DAILYHeatingDegreeDays,DAILYCoolingDegreeDays,DAILYSunrise,DAILYSunset,DAILYWeather,DAILYPrecip,DAILYSnowfall,DAILYSnowDepth,DAILYAverageStationPressure,DAILYAverageSeaLevelPressure,DAILYAverageWindSpeed,DAILYPeakWindSpeed,PeakWindDirection,DAILYSustainedWindSpeed,DAILYSustainedWindDirection,MonthlyMaximumTemp,MonthlyMinimumTemp,MonthlyMeanTemp,MonthlyAverageRH,MonthlyDewpointTemp,MonthlyWetBulbTemp,MonthlyAvgHeatingDegreeDays,MonthlyAvgCoolingDegreeDays,MonthlyStationPressure,MonthlySeaLevelPressure,MonthlyAverageWindSpeed,MonthlyTotalSnowfall,MonthlyDeptFromNormalMaximumTemp,MonthlyDeptFromNormalMinimumTemp,MonthlyDeptFromNormalAverageTemp,MonthlyDeptFromNormalPrecip,MonthlyTotalLiquidPrecip,MonthlyGreatestPrecip,MonthlyGreatestPrecipDate,MonthlyGreatestSnowfall,MonthlyGreatestSnowfallDate,MonthlyGreatestSnowDepth,MonthlyGreatestSnowDepthDate,MonthlyDaysWithGT90Temp,MonthlyDaysWithLT32Temp,MonthlyDaysWithGT32Temp,MonthlyDaysWithLT0Temp,MonthlyDaysWithGT001Precip,MonthlyDaysWithGT010Precip,MonthlyDaysWithGT1Snow,MonthlyMaxSeaLevelPressureValue,MonthlyMaxSeaLevelPressureDate,MonthlyMaxSeaLevelPressureTime,MonthlyMinSeaLevelPressureValue,MonthlyMinSeaLevelPressureDate,MonthlyMinSeaLevelPressureTime,MonthlyTotalHeatingDegreeDays,MonthlyTotalCoolingDegreeDays,MonthlyDeptFromNormalHeatingDD,MonthlyDeptFromNormalCoolingDD,MonthlyTotalSeasonToDateHeatingDD,MonthlyTotalSeasonToDateCoolingDD
0,WBAN:12918,HOUSTON WILLIAM P HOBBY AIRPORT TX US,13.4,29.63806,-95.28194,2013-09-01 00:53,FM-15,FEW:02 30,10.00,NaN,77,25,73.0,22.6,71,21.7,82.0,3,170,NaN,29.83,NaN,NaN,29.89,0.00,29.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,558,1844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
1,WBAN:12918,HOUSTON WILLIAM P HOBBY AIRPORT TX US,13.4,29.63806,-95.28194,2013-09-01 01:53,FM-15,SCT:04 25,10.00,NaN,77,25,73.0,22.6,71,21.7,82.0,6,210,NaN,29.83,NaN,NaN,29.89,0.00,29.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,558,1844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
2,WBAN:12918,HOUSTON WILLIAM P HOBBY AIRPORT TX US,13.4,29.63806,-95.28194,2013-09-01 02:53,FM-15,FEW:02 25,10.00,NaN,76,24.4,72.0,22.4,71,21.7,85.0,3,200,NaN,29.83,5.0,NaN,29.89,0.00,29.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,558,1844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
3,WBAN:12918,HOUSTON WILLIAM P HOBBY AIRPORT TX US,13.4,29.63806,-95.28194,2013-09-01 03:53,FM-15,FEW:02 25,10.00,NaN,75,23.9,72.0,22.2,71,21.7,88.0,3,200,NaN,29.83,NaN,NaN,29.89,0.00,29.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,558,1844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
4,WBAN:12918,HOUSTON WILLIAM P HOBBY AIRPORT TX US,13.4,29.63806,-95.28194,2013-09-01 04:53,FM-15,SCT:04 29,10.00,NaN,75,23.9,72.0,22.2,71,21.7,88.0

### We will filter for fields that pertain to our project.

In [10]:
#Select the weather parameters which affect flight status: Visibility, Temperature, Wind Speed, Precipitation
df_weather = df_weather[['STATION_NAME','DATE','HOURLYVISIBILITY','HOURLYDRYBULBTEMPC','HOURLYWindSpeed','HOURLYPrecip']].copy()

In [11]:
df_weather.head(5)

,STATION_NAME,DATE,HOURLYVISIBILITY,HOURLYDRYBULBTEMPC,HOURLYWindSpeed,HOURLYPrecip
0,HOUSTON WILLIAM P HOBBY AIRPORT TX US,2013-09-01 00:53,10.00,25,3,0.00
1,HOUSTON WILLIAM P HOBBY AIRPORT TX US,2013-09-01 01:53,10.00,25,6,0.00
2,HOUSTON WILLIAM P HOBBY AIRPORT TX US,2013-09-01 02:53,10.00,24.4,3,0.00
3,HOUSTON WILLIAM P HOBBY AIRPORT TX US,2013-09-01 03:53,10.00,23.9,3,0.00
4,HOUSTON WILLIAM P HOBBY AIRPORT TX US,2013-09-01 04:53,10.00,23.9,0,0.00


## Preparing the Weather dataset:
* Replace Long format station name with IATA codes #Need to fix an automated way to do this
* Fix incorrect and erroneous data, such as characters in temperature fields, etc
* Convert Timestamp into YEAR, MONTH, DAY_OF_MONTH and HOUR
* Remove duplicates from the dataset, i.e. multiple entries from same STATION for same HOUR on a particular Date
* Replace NaN with 0 in HOURLY_PRECIP
* Replace NaN with Mean Visibility in HOURLYVISIBILITY

### We will first replace the long format names to abbreviations.

In [12]:
#Replacing Long Format Station Names with IATA Airport codes
df_weather['STATION_NAME'].replace('ATLANTA HARTSFIELD INTERNATIONAL AIRPORT GA US','ATL',inplace=True)
df_weather['STATION_NAME'].replace('CHICAGO OHARE INTERNATIONAL AIRPORT IL US','ORD',inplace=True)
df_weather['STATION_NAME'].replace('DAL FTW WSCMO AIRPORT TX US','DFW',inplace=True)
df_weather['STATION_NAME'].replace('DENVER INTERNATIONAL AIRPORT CO US','DEN',inplace=True)
df_weather['STATION_NAME'].replace('LOS ANGELES INTERNATIONAL AIRPORT CA US','LAX',inplace=True)
df_weather['STATION_NAME'].replace('SAN FRANCISCO INTERNATIONAL AIRPORT CA US','SFO',inplace=True)
df_weather['STATION_NAME'].replace('PHOENIX SKY HARBOR INTERNATIONAL AIRPORT AZ US','PHX',inplace=True)
df_weather['STATION_NAME'].replace('HOUSTON INTERCONTINENTAL AIRPORT TX US','IAH',inplace=True)
df_weather['STATION_NAME'].replace('LAS VEGAS MCCARRAN INTERNATIONAL AIRPORT NV US','LAS',inplace=True)
df_weather['STATION_NAME'].replace('MINNEAPOLIS ST PAUL INTERNATIONAL AIRPORT MN US','MSP',inplace=True)
df_weather['STATION_NAME'].replace('DETROIT METROPOLITAN AIRPORT MI US','DTW',inplace=True)
df_weather['STATION_NAME'].replace('SEATTLE TACOMA INTERNATIONAL AIRPORT WA US','SEA',inplace=True)
df_weather['STATION_NAME'].replace('ORLANDO INTERNATIONAL AIRPORT FL US','MCO',inplace=True)
df_weather['STATION_NAME'].replace('BOSTON MA US','BOS',inplace=True)
df_weather['STATION_NAME'].replace('CHARLOTTE DOUGLAS AIRPORT NC US','CLT',inplace=True)
df_weather['STATION_NAME'].replace('NEWARK LIBERTY INTERNATIONAL AIRPORT NJ US','EWR',inplace=True)
df_weather['STATION_NAME'].replace('SALT LAKE CITY INTERNATIONAL AIRPORT UT US','SLC',inplace=True)
df_weather['STATION_NAME'].replace('LA GUARDIA AIRPORT NY US','LGA',inplace=True)
df_weather['STATION_NAME'].replace('JFK INTERNATIONAL AIRPORT NY US','JFK',inplace=True)
df_weather['STATION_NAME'].replace('BALTIMORE WASHINGTON INTERNATIONAL AIRPORT MD US','BWI',inplace=True)
df_weather['STATION_NAME'].replace('CHICAGO MIDWAY AIRPORT IL US','MDW',inplace=True)
df_weather['STATION_NAME'].replace('MIAMI INTERNATIONAL AIRPORT FL US','MIA',inplace=True)
df_weather['STATION_NAME'].replace('SAN DIEGO INTERNATIONAL AIRPORT CA US','SAN',inplace=True)
df_weather['STATION_NAME'].replace('WASHINGTON REAGAN NATIONAL AIRPORT VA US','DCA',inplace=True)
df_weather['STATION_NAME'].replace('FORT LAUDERDALE HOLLYWOOD INTERNATIONAL AIRPORT FL US','FLL',inplace=True)
df_weather['STATION_NAME'].replace('PHILADELPHIA INTERNATIONAL AIRPORT PA US','PHL',inplace=True)
df_weather['STATION_NAME'].replace('TAMPA INTERNATIONAL AIRPORT FL US','TPA',inplace=True)
df_weather['STATION_NAME'].replace('DALLAS FAA AIRPORT TX US','DAL',inplace=True)
df_weather['STATION_NAME'].replace('HOUSTON WILLIAM P HOBBY AIRPORT TX US','HOU',inplace=True)
df_weather['STATION_NAME'].replace('PORTLAND INTERNATIONAL AIRPORT OR US','PDX',inplace=True)
df_weather['STATION_NAME'].replace('NASHVILLE INTERNATIONAL AIRPORT TN US','BNA',inplace=True)
df_weather['STATION_NAME'].replace('ST LOUIS LAMBERT INTERNATIONAL AIRPORT MO US','STL',inplace=True)
df_weather['STATION_NAME'].replace('WASHINGTON DULLES INTERNATIONAL AIRPORT VA US','IAD',inplace=True)
df_weather['STATION_NAME'].replace('HONOLULU INTERNATIONAL AIRPORT HI US','HNL',inplace=True)
df_weather['STATION_NAME'].replace('OAKLAND METROPOLITAN INTERNATIONAL AIRPORT CA US','OAK',inplace=True)
df_weather['STATION_NAME'].replace('AUSTIN BERGSTROM INTERNATIONAL AIRPORT TX US','AUS',inplace=True)
df_weather['STATION_NAME'].replace('KANSAS CITY INTERNATIONAL AIRPORT MO US','MCI',inplace=True)
df_weather['STATION_NAME'].replace('NEW ORLEANS INTERNATIONAL AIRPORT LA US','MSY',inplace=True)
df_weather['STATION_NAME'].replace('SAN JOSE CA US','SJC',inplace=True)
df_weather['STATION_NAME'].replace('SACRAMENTO METROPOLITAN AIRPORT CA US','SMF',inplace=True)
df_weather['STATION_NAME'].replace('SANTA ANA JOHN WAYNE AIRPORT CA US','SNA',inplace=True)
df_weather['STATION_NAME'].replace('CLEVELAND HOPKINS INTERNATIONAL AIRPORT OH US','CLE',inplace=True)
df_weather['STATION_NAME'].replace('RALEIGH AIRPORT NC US','RDU',inplace=True)
df_weather['STATION_NAME'].replace('MILWAUKEE MITCHELL INTERNATIONAL AIRPORT WI US','MKE',inplace=True)
df_weather['STATION_NAME'].replace('SAN ANTONIO INTERNATIONAL AIRPORT TX US','SAT',inplace=True)
df_weather['STATION_NAME'].replace('INDIANAPOLIS INTERNATIONAL AIRPORT IN US','IND',inplace=True)
df_weather['STATION_NAME'].replace('FORT MYERS SW FLORIDA REGIONAL AIRPORT FL US','RSW',inplace=True)
df_weather['STATION_NAME'].replace('PITTSBURGH ASOS PA US','PIT',inplace=True)
df_weather['STATION_NAME'].replace('SAN JUAN L M MARIN INTERNATIONAL AIRPORT US','SJU',inplace=True)
df_weather['STATION_NAME'].replace('PORT COLUMBUS INTERNATIONAL AIRPORT OH US','CMH',inplace=True)

### This function will be to fix incorrect data, such as characters in temperature fields.

In [13]:
def convert(x):
        try:
            if str(x)[-1].isalpha():
                return(float(str(x)[:-1]))
            else:
                return(float(str(x)))
        except:
            return(np.nan)

In [14]:
df_weather['HOURLYVISIBILITY'] = df_weather['HOURLYVISIBILITY'].apply(lambda x: convert(x))
df_weather['HOURLYDRYBULBTEMPC'] = df_weather['HOURLYDRYBULBTEMPC'].apply(lambda x: convert(x))
df_weather['HOURLYWindSpeed'] = df_weather['HOURLYWindSpeed'].apply(lambda x: convert(x))
df_weather['HOURLYPrecip'] = df_weather['HOURLYPrecip'].apply(lambda x: convert(x))

### We will extract the year, month, date, and hour from our weather dataframe.

In [15]:
df_weather['DATE'] = pd.to_datetime(df_weather['DATE'])

df_weather['YEAR']= df_weather['DATE'].apply(lambda time: time.year)
df_weather['MONTH']= df_weather['DATE'].apply(lambda time: time.month)
df_weather['DAY_OF_MONTH']= df_weather['DATE'].apply(lambda time: time.day)
df_weather['HOUR']= df_weather['DATE'].apply(lambda time: time.hour)

### We will remove duplicates in our dataframe

In [16]:
df_weather.drop_duplicates(['STATION_NAME','YEAR','MONTH','DAY_OF_MONTH','HOUR'],inplace=True)
df_weather.drop('DATE',axis = 1,inplace=True)

### We will replace NA values with 0 for hourly precipitation and replace NA values with the average in hourly visibility.

In [17]:
df_weather['HOURLYPrecip'].fillna(value=0,inplace=True)

df_weather['HOURLYVISIBILITY'].fillna(df_weather['HOURLYVISIBILITY'].mean(),inplace=True)

In [18]:
df_weather.head(5)

,STATION_NAME,HOURLYVISIBILITY,HOURLYDRYBULBTEMPC,HOURLYWindSpeed,HOURLYPrecip,YEAR,MONTH,DAY_OF_MONTH,HOUR
0,HOU,10.0,25.0,3.0,0.0,2013,9,1,0
1,HOU,10.0,25.0,6.0,0.0,2013,9,1,1
2,HOU,10.0,24.4,3.0,0.0,2013,9,1,2
3,HOU,10.0,23.9,3.0,0.0,2013,9,1,3
4,HOU,10.0,23.9,0.0,0.0,2013,9,1,4


### We will check for missing values

In [19]:
df_weather.isnull().sum()

STATION_NAME           0
HOURLYVISIBILITY       0
HOURLYDRYBULBTEMPC     8
HOURLYWindSpeed       13
HOURLYPrecip           0
YEAR                   0
MONTH                  0
DAY_OF_MONTH           0
HOUR                   0
dtype: int64

### Next, we will calculate the average weather values for each station, for example, the annual mean temperature. We will also create two dataframes, one for origin and one for the destination. 

In [20]:
df_avg_DEP = df_weather.groupby('STATION_NAME').mean()
df_avg_DEP.drop(['YEAR','MONTH','DAY_OF_MONTH','HOUR'],axis = 1,inplace=True)
df_avg_DEP.reset_index(drop=False,inplace=True)
df_avg_DEP.rename(index=str, columns={"STATION_NAME": "ORIGIN"},inplace=True)
df_avg_DEP.rename(index=str, columns={"HOURLYVISIBILITY": "DEP_AVG_HOURLYVISIBILITY"},inplace=True)
df_avg_DEP.rename(index=str, columns={"HOURLYDRYBULBTEMPC": "DEP_AVG_HOURLYDRYBULBTEMPC"},inplace=True)
df_avg_DEP.rename(index=str, columns={"HOURLYWindSpeed": "DEP_AVG_HOURLYWindSpeed"},inplace=True)
df_avg_DEP.rename(index=str, columns={"HOURLYPrecip": "DEP_AVG_HOURLYPrecip"},inplace=True)

df_avg_ARR = df_weather.groupby('STATION_NAME').mean()
df_avg_ARR.drop(['YEAR','MONTH','DAY_OF_MONTH','HOUR'],axis = 1,inplace=True)
df_avg_ARR.reset_index(drop=False,inplace=True)
df_avg_ARR.rename(index=str, columns={"STATION_NAME": "DEST"},inplace=True)
df_avg_ARR.rename(index=str, columns={"HOURLYVISIBILITY": "ARR_AVG_HOURLYVISIBILITY"},inplace=True)
df_avg_ARR.rename(index=str, columns={"HOURLYDRYBULBTEMPC": "ARR_AVG_HOURLYDRYBULBTEMPC"},inplace=True)
df_avg_ARR.rename(index=str, columns={"HOURLYWindSpeed": "ARR_AVG_HOURLYWindSpeed"},inplace=True)
df_avg_ARR.rename(index=str, columns={"HOURLYPrecip": "ARR_AVG_HOURLYPrecip"},inplace=True)

In [21]:
df_weather_origin = df_weather.copy()
df_weather_dest = df_weather.copy()
del df_weather

#Rename the Columns, add DEP_ to each column name and STATION_NAME to ORIGIN
df_weather_origin.rename(index=str, columns={"STATION_NAME": "ORIGIN"},inplace=True)
df_weather_origin.rename(index=str, columns={"HOURLYVISIBILITY": "DEP_HOURLYVISIBILITY"},inplace=True)
df_weather_origin.rename(index=str, columns={"HOURLYDRYBULBTEMPC": "DEP_HOURLYDRYBULBTEMPC"},inplace=True)
df_weather_origin.rename(index=str, columns={"HOURLYWindSpeed": "DEP_HOURLYWindSpeed"},inplace=True)
df_weather_origin.rename(index=str, columns={"HOURLYPrecip": "DEP_HOURLYPrecip"},inplace=True)
df_weather_origin.rename(index=str, columns={"HOUR": "DEP_HOUR"},inplace=True)

#Rename the Columns, add ARR_ to each column name and STATION_NAME to DEST
df_weather_dest.rename(index=str, columns={"STATION_NAME": "DEST"},inplace=True)
df_weather_dest.rename(index=str, columns={"HOURLYVISIBILITY": "ARR_HOURLYVISIBILITY"},inplace=True)
df_weather_dest.rename(index=str, columns={"HOURLYDRYBULBTEMPC": "ARR_HOURLYDRYBULBTEMPC"},inplace=True)
df_weather_dest.rename(index=str, columns={"HOURLYWindSpeed": "ARR_HOURLYWindSpeed"},inplace=True)
df_weather_dest.rename(index=str, columns={"HOURLYPrecip": "ARR_HOURLYPrecip"},inplace=True)
df_weather_dest.rename(index=str, columns={"HOUR": "ARR_HOUR"},inplace=True)

### We will create four new columns from the time columns, for departure and arrival of actual hours and computer reservation hours (CRS).

In [22]:
df["DEP_HOUR"] = df["DEP_TIME"].apply(lambda x:x//100)
df =df[np.isfinite(df['DEP_HOUR'])]
df["DEP_HOUR"]=df["DEP_HOUR"].astype('int64')

df["ARR_HOUR"] =  df["ARR_TIME"].apply(lambda x: x//100)
df =df[np.isfinite(df["ARR_HOUR"])]
df["ARR_HOUR"]=df["ARR_HOUR"].astype('int64')

df["CRS_DEP_HOUR"] = df["CRS_DEP_TIME"].apply(lambda x:x//100)
df =df[np.isfinite(df['CRS_DEP_HOUR'])]
df["CRS_DEP_HOUR"]=df["CRS_DEP_HOUR"].astype('int64')

df["CRS_ARR_HOUR"] =  df["CRS_ARR_TIME"].apply(lambda x: x//100)
df =df[np.isfinite(df["CRS_ARR_HOUR"])]
df["CRS_ARR_HOUR"]=df["CRS_ARR_HOUR"].astype('int64')

In [23]:
print(df_weather_origin.shape)
print(df.shape)

(99351, 9)
(299011, 22)


### We can see that our weather location is only linked to 4 cities, so we will filter our airline dataframe for only the labeled sities. 

In [24]:
df_weather_origin.groupby("ORIGIN").count()

,DEP_HOURLYVISIBILITY,DEP_HOURLYDRYBULBTEMPC,DEP_HOURLYWindSpeed,DEP_HOURLYPrecip,YEAR,MONTH,DAY_OF_MONTH,DEP_HOUR
ORIGIN,,,,,,,,
BNA,20442,20442,20441,20442,20442,20442,20442,20442
DAL,26303,26303,26299,26303,26303,26303,26303,26303
HOU,26304,26303,26301,26304,26304,26304,26304,26304
STL,26302,26295,26297,26302,26302,26302,26302,26302


In [25]:
df=df[df["ORIGIN"].apply(lambda x: x in ["BNA","DAL","HOU","STL"])]
df=df[df["DEST"].apply(lambda x: x in ["BNA","DAL","HOU","STL"])]

In [26]:
print(df["YEAR"].dtype,df["YEAR"].dtype,df["DAY_OF_MONTH"].dtype,df["DEP_HOUR"].dtype)
print(df_weather_origin["YEAR"].dtype,df_weather_origin["YEAR"].dtype,df_weather_origin["DAY_OF_MONTH"].dtype,df_weather_origin["DEP_HOUR"].dtype)

int64 int64 int64 int64
int64 int64 int64 int64


### We will join the weather for origin and destination Airports for each flight in the data frame

In [27]:
df= pd.merge(df, df_weather_origin, on=['ORIGIN','YEAR','MONTH','DAY_OF_MONTH','DEP_HOUR'], how='left')
df = pd.merge(df, df_weather_dest, on=['DEST','YEAR','MONTH','DAY_OF_MONTH','ARR_HOUR'], how='left')

In [28]:
df.head(5)

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,UNIQUE_CARRIER,FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,DEP_HOUR,ARR_HOUR,CRS_DEP_HOUR,CRS_ARR_HOUR,DEP_HOURLYVISIBILITY,DEP_HOURLYDRYBULBTEMPC,DEP_HOURLYWindSpeed,DEP_HOURLYPrecip,ARR_HOURLYVISIBILITY,ARR_HOURLYDRYBULBTEMPC,ARR_HOURLYWindSpeed,ARR_HOURLYPrecip
0,2013,9,1,7,WN,60,BNA,HOU,1830,1845.0,15.0,2025,2032.0,7.0,115.0,107.0,93.0,670.0,18,20,18,20,9.360781,26.1,0.0,0.0,10.0,27.8,6.0,0.0
1,2013,9,1,7,WN,64,BNA,HOU,840,840.0,0.0,1035,1025.0,-10.0,115.0,105.0,90.0,670.0,8,10,8,10,10.000000,21.7,6.0,0.0,10.0,31.7,9.0,0.0
2,2013,9,1,7,WN,912,BNA,HOU,1000,958.0,-2.0,1155,1144.0,-11.0,115.0,106.0,93.0,670.0,9,11,10,11,9.360781,21.7,6.0,0.0,10.0,32.8,9.0,0.0
3,2013,9,1,7,WN,54,BNA,STL,1545,1542.0,-3.0,1650,1636.0,-14.0,65.0,54.0,44.0,272.0,15,16,15,16,9.360781,27.2,5.0,0.0,10.0,31.1,6.0,0.0
4,2013,9,1,7,WN,19,DAL,HOU,1100,1100.0,0.0,1200,1155.0,-5.0,60.0,55.0,40.0,239.0,11,11,11,12,10.000000,37.2,3.0,0.0,10.0,32.8,9.0,0.0


In [29]:
df.shape

(2354, 30)

### We will also join the average for origin and destination Airports for each flight in the data frame

In [30]:
df = pd.merge(df,df_avg_DEP,how='left',on='ORIGIN')
df = pd.merge(df,df_avg_ARR,how='left',on='DEST')

In [31]:
df.shape

(2354, 38)

In [32]:
df.head(5)

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,UNIQUE_CARRIER,FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,DEP_HOUR,ARR_HOUR,CRS_DEP_HOUR,CRS_ARR_HOUR,DEP_HOURLYVISIBILITY,DEP_HOURLYDRYBULBTEMPC,DEP_HOURLYWindSpeed,DEP_HOURLYPrecip,ARR_HOURLYVISIBILITY,ARR_HOURLYDRYBULBTEMPC,ARR_HOURLYWindSpeed,ARR_HOURLYPrecip,DEP_AVG_HOURLYVISIBILITY,DEP_AVG_HOURLYDRYBULBTEMPC,DEP_AVG_HOURLYWindSpeed,DEP_AVG_HOURLYPrecip,ARR_AVG_HOURLYVISIBILITY,ARR_AVG_HOURLYDRYBULBTEMPC,ARR_AVG_HOURLYWindSpeed,ARR_AVG_HOURLYPrecip
0,2013,9,1,7,WN,60,BNA,HOU,1830,1845.0,15.0,2025,2032.0,7.0,115.0,107.0,93.0,670.0,18,20,18,20,9.360781,26.1,0.0,0.0,10.0,27.8,6.0,0.0,9.381034,15.194663,6.236926,0.001916,9.459709,21.153017,7.272993,0.003151
1,2013,9,1,7,WN,64,BNA,HOU,840,840.0,0.0,1035,1025.0,-10.0,115.0,105.0,90.0,670.0,8,10,8,10,10.000000,21.7,6.0,0.0,10.0,31.7,9.0,0.0,9.381034,15.194663,6.236926,0.001916,9.459709,21.153017,7.272993,0.003151
2,2013,9,1,7,WN,912,BNA,HOU,1000,958.0,-2.0,1155,1144.0,-11.0,115.0,106.0,93.0,670.0,9,11,10,11,9.360781,21.7,6.0,0.0,10.0,32.8,9.0,0.0,9.381034,15.194663,6.236926,0.001916,9.459709,21.153017,7.272993,0.003151
3,2013,9,1,7,WN,54,BNA,STL,1545,1542.0,-3.0,1650,1636.0,-14.0,65.0,54.0,44.0,272.0,15,16,15,16,9.360781,27.2,5.0,0.0,10.0,31.1,6.0,0.0,9.381034,15.194663,6.236926,0.001916,9.238202,14.412980,8.346960,0.001833
4,2013,9,1,7,WN,19,DAL,HOU,1100,1100.0,0.0,1200,1155.0,-5.0,60.0,55.0,40.0,239.0,11,11,11,12,10.000000,37.2,3.0,0.0,10.0,32.8,9.0,0.0,9.368683,19.650112,9.433743,0.001829,9.459709,21.153017,7.272993,0.003151


In [33]:
df.head(2)

,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,UNIQUE_CARRIER,FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,DEP_HOUR,ARR_HOUR,CRS_DEP_HOUR,CRS_ARR_HOUR,DEP_HOURLYVISIBILITY,DEP_HOURLYDRYBULBTEMPC,DEP_HOURLYWindSpeed,DEP_HOURLYPrecip,ARR_HOURLYVISIBILITY,ARR_HOURLYDRYBULBTEMPC,ARR_HOURLYWindSpeed,ARR_HOURLYPrecip,DEP_AVG_HOURLYVISIBILITY,DEP_AVG_HOURLYDRYBULBTEMPC,DEP_AVG_HOURLYWindSpeed,DEP_AVG_HOURLYPrecip,ARR_AVG_HOURLYVISIBILITY,ARR_AVG_HOURLYDRYBULBTEMPC,ARR_AVG_HOURLYWindSpeed,ARR_AVG_HOURLYPrecip
0,2013,9,1,7,WN,60,BNA,HOU,1830,1845.0,15.0,2025,2032.0,7.0,115.0,107.0,93.0,670.0,18,20,18,20,9.360781,26.1,0.0,0.0,10.0,27.8,6.0,0.0,9.381034,15.194663,6.236926,0.001916,9.459709,21.153017,7.272993,0.003151
1,2013,9,1,7,WN,64,BNA,HOU,840,840.0,0.0,1035,1025.0,-10.0,115.0,105.0,90.0,670.0,8,10,8,10,10.000000,21.7,6.0,0.0,10.0,31.7,9.0,0.0,9.381034,15.194663,6.236926,0.001916,9.459709,21.153017,7.272993,0.003151


### In our new data frame, we will create a target column that will label any flight as True that has passed 8 minutes on their expected arrival time. 

In [34]:
df["ARRIVIAL_DELAYED"] =  df["ARR_DELAY"].apply(lambda x: "YES" if x > 8 else "NO")

In [35]:
df["ARRIVIAL_DELAYED"].value_counts()

NO     1694
YES     660
Name: ARRIVIAL_DELAYED, dtype: int64

### We will then drop the ARR_Delay column as well as any column that involves any air or arrival time data. We will remove these additional data because our model would rely to heavily on the data for prediction, and because some of the information is to closely related to the target column. 

In [36]:
df.drop(['ARR_DELAY'],axis=1,inplace=True)

In [37]:
df.drop(['ARR_TIME','AIR_TIME','ACTUAL_ELAPSED_TIME','ARR_HOUR'],axis=1,inplace=True)

### Save the DataFrames as a .csv file in order to import to BigML

In [38]:
df.to_csv('data/Airline+Weather_data.csv',index=False)

### Save our BigML Username and Api Key to our environment to access the API

In [39]:
import os
os.environ['BIGML_USERNAME'] = "EFETOROS"
os.environ['BIGML_API_KEY'] = "7e5fc6a649fd0f8517fc8ecf2ebd30151c5d4fb4"

### Creat our main API object that all the main functions will utilize.

In [40]:
from bigml.api import BigML
api = BigML()

### Importing Data to BigML
In order to start a BigML workflow, a source object has to be created. The API function that creates a source is <code>create_source</code>. The method's inputs will be a file path to the csv it will be converting. The source will be created from the csv files written by <code>to_csv</code> from before. 

In [41]:
source = api.create_source('data/Airline+Weather_data.csv')

BigML's <code>ok</code> method is called in order to assure that an object is created and will wait if it is not done being completed.

In [42]:
api.ok(source)

True

### Creating a Dataset
BigML will use the newly created source to create datasets which will enable the API to perform many more operations. In order to create a dataset, the API calls the function <code>create_dataset</code>. The method will take the source created by the API as an input.

In [43]:
origin_dataset = api.create_dataset(source)

### Test-Train Split
Since we want our data to stay in the form of BigML's datasets, the test-train split of the data will be done through BigML's API. This form will allow for the API's computations. The test-train split will be created by the function <code>create_dataset</code> mentioned before. However, it will take advantage of the more available inputs of the function. Many BigML API functions take in a dictionary with many fields as an additional input. These fields allow for much manipulation of the original function's outcome. In a test-train split, the field of sample_rate will allow for the choosing of the percentage of data being sampled. The train dataset will have out_of_bag field set to False and the test dataset will have it set to True. Since the test out_of_bag is set to True, its size will be 20% when its sample rate is 80%.

In [44]:
origin_dataset = api.create_dataset(source)
train_dataset = api.create_dataset(
    origin_dataset, {"name": "Flight Delay | Training",
                     "sample_rate": 0.8, "seed": "my seed"})
test_dataset = api.create_dataset(
    origin_dataset, {"name": "Flight Delay | Test",
                     "sample_rate": 0.8, "seed": "my seed",
                     "out_of_bag": True})
api.ok(train_dataset)
api.ok(test_dataset)

True

### Creating Ensembles
 BigML's API allows for the creation of many models. For this dataset, Ensembles will be used. The BigML API will use the method <code>create_ensemble</code>. As stated before, the method takes in additional inputs for different use cases and manipulation of the function. We will create to ensembles.

1) In the first ensemble, we will balance the weights of the objective field, since there are more flights not delayed than delayed.


In [45]:
ensemble_1 = api.create_ensemble(train_dataset, { \
    "balance_objective" : True})
api.ok(ensemble_1)

True

2) In the second ensemble, we will also balance the weights of the objective field, but we will also choose to exclude any information that involves exact departure time. For example, we will leave in computer reservation system departure time (CRS_DEP_TIME), since this data is the scheduled time and will be known even before take off, but we will drop departure delay (DEP_DELAY), which will only be known in the exact moment or very close to take off. This model will lose performance, however, it can be used to make predictions if a flight is going to be delayed much before the flight has taken off, on the other hand, the first model heavily relies on departure delay data to identify arrival delay, which might not be very useful. 


In [47]:
ensemble_2 = api.create_ensemble(train_dataset, { \
#     "objective_weights": [["YES", 8], ["NO", 1]],
    "balance_objective" : True,
    "excluded_fields": ["DEP_HOUR","DEP_TIME","DEP_DELAY"]})
api.ok(ensemble_2)

True

This function will be used to retrieve field names from IDs.

In [48]:
def names(field_importance):
    names_of_important_fields = {}
    for keys in field_importance.keys():
        names_of_important_fields[train_dataset['object']["fields"][keys]["name"]] = field_importance[keys]
    sorted_values = sorted(names_of_important_fields.items(), key=lambda kv: kv[1])
    return sorted_values

### Below will be a list of ordered field importance for both models. We can see that the first model put a lot of weight on departure delay.

#### BigML API objects, such as ensemble_1, are nested dictionaries and will have many information within.

In [49]:
field_importance_ensemble_1 = ensemble_1["object"]["importance"]

In [50]:
names(field_importance_ensemble_1)

[('CRS_DEP_HOUR', 0.00054),
 ('DEP_AVG_HOURLYWindSpeed', 0.00061),
 ('ARR_AVG_HOURLYWindSpeed', 0.00068),
 ('DEP_HOUR', 0.00083),
 ('ARR_AVG_HOURLYPrecip', 0.001),
 ('ORIGIN', 0.00101),
 ('DEST', 0.00127),
 ('DEP_AVG_HOURLYPrecip', 0.00238),
 ('DISTANCE', 0.00365),
 ('ARR_AVG_HOURLYDRYBULBTEMPC', 0.00556),
 ('CRS_ARR_HOUR', 0.0056),
 ('ARR_AVG_HOURLYVISIBILITY', 0.00764),
 ('DEP_HOURLYPrecip', 0.00785),
 ('DEP_AVG_HOURLYDRYBULBTEMPC', 0.00972),
 ('CRS_ELAPSED_TIME', 0.01719),
 ('DEP_TIME', 0.01841),
 ('DAY_OF_WEEK', 0.01927),
 ('ARR_HOURLYVISIBILITY', 0.01959),
 ('ARR_HOURLYWindSpeed', 0.02373),
 ('ARR_HOURLYPrecip', 0.02515),
 ('DEP_HOURLYVISIBILITY', 0.03278),
 ('CRS_ARR_TIME', 0.03327),
 ('FL_NUM', 0.03361),
 ('DEP_HOURLYWindSpeed', 0.03791),
 ('ARR_HOURLYDRYBULBTEMPC', 0.04883),
 ('DEP_HOURLYDRYBULBTEMPC', 0.05087),
 ('CRS_DEP_TIME', 0.0593),
 ('DAY_OF_MONTH', 0.06681),
 ('DEP_DELAY', 0.46495)]

In [51]:
field_importance_ensemble_2 = ensemble_2["object"]["importance"]

In [52]:
names(field_importance_ensemble_2)

[('DEST', 0.0003),
 ('DEP_AVG_HOURLYPrecip', 0.00038),
 ('ARR_AVG_HOURLYDRYBULBTEMPC', 0.00125),
 ('DEP_AVG_HOURLYWindSpeed', 0.00129),
 ('CRS_DEP_HOUR', 0.00131),
 ('DEP_AVG_HOURLYDRYBULBTEMPC', 0.00149),
 ('ORIGIN', 0.00189),
 ('ARR_AVG_HOURLYWindSpeed', 0.00455),
 ('ARR_AVG_HOURLYVISIBILITY', 0.00652),
 ('ARR_HOURLYVISIBILITY', 0.00932),
 ('DEP_AVG_HOURLYVISIBILITY', 0.00939),
 ('ARR_AVG_HOURLYPrecip', 0.00967),
 ('DISTANCE', 0.0143),
 ('ARR_HOURLYPrecip', 0.01912),
 ('DEP_HOURLYPrecip', 0.02235),
 ('DEP_HOURLYVISIBILITY', 0.02954),
 ('CRS_ELAPSED_TIME', 0.03296),
 ('ARR_HOURLYWindSpeed', 0.05649),
 ('FL_NUM', 0.06089),
 ('DEP_HOURLYWindSpeed', 0.06837),
 ('DEP_HOURLYDRYBULBTEMPC', 0.07274),
 ('CRS_ARR_HOUR', 0.08578),
 ('CRS_DEP_TIME', 0.0899),
 ('DAY_OF_MONTH', 0.09274),
 ('CRS_ARR_TIME', 0.09308),
 ('DAY_OF_WEEK', 0.10536),
 ('ARR_HOURLYDRYBULBTEMPC', 0.10905)]

### Creating Evaluations
BigML's API allows for the creation of evaluations for specific models. The BigML API will use the method <code>create_evaluation</code>, and will take in the model of interest and the test_dataset.

In [53]:
evaluation_1 = api.create_evaluation(ensemble_1, test_dataset)
api.ok(evaluation_1)

True

In [54]:
evaluation_2 = api.create_evaluation(ensemble_2, test_dataset)
api.ok(evaluation_2)

True

### Below are performance metrics for both models, as seen, the second model is not as accurate, however it may prove to be more useful. For example, an airline can mark a plane that has a higher chance of being delayed before departure, and allocate resources to hopefully save further delay expensives.


In [55]:
print("Accuracy: ",evaluation_1["object"]["result"]["model"]["accuracy"])
print("Recall: ",evaluation_1["object"]["result"]["model"]["average_recall"])
print("Precision: ",evaluation_1["object"]["result"]["model"]["average_precision"])

Accuracy:  0.87261
Recall:  0.83455
Precision:  0.86642


In [56]:
print("Accuracy: ",evaluation_2["object"]["result"]["model"]["accuracy"])
print("Recall: ",evaluation_2["object"]["result"]["model"]["average_recall"])
print("Precision: ",evaluation_2["object"]["result"]["model"]["average_precision"])

Accuracy:  0.75159
Recall:  0.64931
Precision:  0.74032
